# CÁLCULO DE VIGA PROTENDIDA - VP43

**OBRA:** SUPERMAGO Porto Alegre

**Cliente:** JD Construções

**Data:** 02/03/2023

**Cálculo:** Gabriel Garcia

<img src="img/logoRS.jpg" alt="RS Engenharia & Arquitetura" width="200px"/>

Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [1]:
import handcalcs.render
import pandas as pd
import numpy as np
from math import floor,ceil,sqrt,exp
from numpy import log, pi
from ggcp import calcpsi, linha_neutra_cp_ret
from scipy.optimize import fsolve


# Informações Iniciais:

## Definição do EPS:

## Geometria da Viga:

In [2]:
%%render
# Geometria da Viga:

h = 105
b = 40
h_eps = 0
e_eps = 0
A_c = b*h - e_eps*h_eps
I_c = (b*h**3/12 - e_eps*(h_eps**3)/12)
y_i = 53.1
y_s = h-y_i
W_i = I_c/(y_i)
W_s = I_c/(y_s)


Vao = 1075

## Número de seções para dividir o vão:
# Escolher número impar
nS = 21

<IPython.core.display.Latex object>

## Propriedades dos Materiais:

In [3]:
%%render
# Propriedades dos Materiais:

f_ck = 4
f_ctm = 0.03*(10*f_ck)**(2/3)
f_ptk = 190
f_pyd = 148.7
f_yk = 50
f_cj = 2.8

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [4]:
%%render 2 short

f_c = f_ck*0.85/1.4
E_ci = 560*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_cj/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

## Carregamentos:

In [5]:
%%render 2 short

Q_t0 = 0.000025*A_c*1.0809
Q_tF = 117.000/100
Q_ELS = 80.61/100

<IPython.core.display.Latex object>

## Esforços no Centro do Vão:

In [6]:
%%render 2 short
M_F = (Q_tF)*Vao**2*0.125
M_0 = (Q_t0)*Vao**2*0.125
M_ELS = Q_ELS*Vao**2/8
V_F = (Q_tF)*Vao/2*1.065

<IPython.core.display.Latex object>

# Deteminação da Força de Protensão:

## Limites de Tensão:

In [7]:
%%render short

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

## Cálculo da Força de Protensão:

In [8]:
%%render short 2

e_p = 35.625

P_1 = (sigma_cc0+M_0/W_i)/(r_0*(1/A_c + e_p/W_i)) 
P_2 = (sigma_ct0-M_0/W_s)/(r_0*(1/A_c - e_p/W_s)) 
P_3 = (sigma_ctF+M_F/W_i)/(r_F*(1/A_c + e_p/W_i))
P_4 = (sigma_ccF-M_F/W_s)/(r_F*(1/A_c - e_p/W_s))

##Adotar valores entre:
P_min = max(P_3, P_4)
P_max = min(P_1, P_2)


<IPython.core.display.Latex object>

## Escolha da Protensão:

In [9]:
%%render 2
# Força de Protensão Adotada:

P = P_max
phi_1 = 1.52
A_p1 = 1.43

## Número de cordoalhas para adotar:
n = floor(P/(A_p1*sigma_p0))

<IPython.core.display.Latex object>

## Ancoragem das Cordoalhas:

### Comprimento de Ancoragem Básico:

In [10]:
%%render short
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     0.7 para situação de má aderência

f_ctd = (0.7*0.03*(f_cj*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### Cálculo do comprimento de Transferência:

In [11]:
%%render short

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.0 * l_bpt2

l_p = (h**2 + (0.6*l_bpt)**2)**(0.5)


<IPython.core.display.Latex object>

#  Esforços nas Seções:

## Momento Fletor e Cortante:

In [12]:
esforcos = np.zeros((nS,3))
esforcos_t0 = np.zeros((nS,3))

# primeiro vetor contem X (cm)
# segundo vetor contem Momentos (kNcm)
# terceiro vetor contém cortante (kN)

for i in range(nS):
    esforcos[i,0] = round(i*(Vao/(nS-1)),1)
    
esforcos[:,1] = ( 0.5*Q_tF*esforcos[:,0]*(Vao-esforcos[:,0]))
esforcos[:,2] = ( 0.5*Q_tF*(Vao-2*esforcos[:,0]))
esforcos_t0[:,0] = esforcos[:,0]
esforcos_t0[:,1] = 0.5*Q_t0*esforcos[:,0]*(Vao-esforcos[:,0])
esforcos_t0[:,2] = 0.5*Q_t0*(Vao-2*esforcos[:,0])

esforcos=np.round(esforcos,decimals=1)
esforcos_t0 = np.round(esforcos_t0,decimals=1)
    
print("ESFORÇOS NO TEMPO FINAL:")
tabela = pd.DataFrame(esforcos)
tabela.rename(columns={0:'Pos (cm)', 1:'M (kNcm)', 2:'V (kN)'},inplace = True)
display(tabela)

ESFORÇOS NO TEMPO FINAL:


,Pos (cm),M (kNcm),V (kN)
0,0.0,0.0,628.9
1,53.8,32140.2,565.9
2,107.5,60843.7,503.1
3,161.2,86173.2,440.3
4,215.0,108166.5,377.3
5,268.8,126773.3,314.4
6,322.5,141968.5,251.6
7,376.2,153789.8,188.7
8,430.0,162249.8,125.8
9,483.8,167323.2,62.8


In [13]:
print("ESFORÇOS NO TEMPO INICIAL:")
tabela2 = pd.DataFrame(esforcos_t0)
tabela2.rename(columns={0:'Pos (cm)', 1:'M (kNcm)', 2:'V (kN)'},inplace = True)
display(tabela2)

ESFORÇOS NO TEMPO INICIAL:


,Pos (cm),M (kNcm),V (kN)
0,0.0,0.0,61.0
1,53.8,3117.7,54.9
2,107.5,5902.1,48.8
3,161.2,8359.1,42.7
4,215.0,10492.6,36.6
5,268.8,12297.5,30.5
6,322.5,13771.5,24.4
7,376.2,14918.2,18.3
8,430.0,15738.8,12.2
9,483.8,16231.0,6.1


# Escalonamento da Protensão:

## Limites de Tensão na Seção S1:

São testados os limites de tensão para t0:

Limitação pela borda superior:

In [14]:
%%render short

M_s1 = round(esforcos_t0[1,1],1) # Momento de peso próprio na seção s1
P_1 = (sigma_cc0+M_s1/W_i)/(r_0*(1/A_c + e_p/W_i))
P_2 = (sigma_ct0-M_s1/W_s)/(r_0*(1/A_c - e_p/W_s))


<IPython.core.display.Latex object>

In [15]:
%%render 1

P_s1 = min(P_1 , P_2)

n_s1 = floor(P_s1/(A_p1*sigma_p0))


<IPython.core.display.Latex object>

## Escolha do Número de Cordoalhas:

In [16]:
# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = np.array([7,10])

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = np.array([37.5,35])

print("Quantidade de Coordoalhas: ",qntCordoalhas)
print("Excentricidade da Protensão: ",e_p_i)

Quantidade de Coordoalhas:  [ 7 10]
Excentricidade da Protensão:  [37.5 35. ]


# Esforços Devidos À Protensão:

In [17]:
A_p = qntCordoalhas*A_p1
W_cp = np.round(I_c/e_p_i ,decimals = 2)
alpha_p0 = E_p / E_cst0
K_p0 = np.round( alpha_p0*A_p*(1/A_c+e_p_i/W_cp) ,decimals=4)
sigma_p = np.round(sigma_p0/(1+K_p0),decimals=2)
N_p = -sigma_p*A_p
M_p = N_p*e_p_i

tabela1=pd.DataFrame([A_p, W_cp, K_p0, sigma_p, N_p, M_p])
tabela1=tabela1.transpose()
tabela1.rename(columns={0:'Ap',1:'Wcp',2:'Kp0',3:'Sigma,p',4:'Np',5:'Mp'},inplace = True)
display(tabela1)

,Ap,Wcp,Kp0,"Sigma,p",Np,Mp
0,10.01,102900.0,0.0468,145.20,-1453.452,-54504.45
1,14.30,110250.0,0.0616,143.18,-2047.474,-71661.59


# Escalonamento da Força de Protensão:

In [18]:
M_g = np.round(W_s*( (N_p/A_c) -(M_p/W_s) + sigma_ct0),decimals=1)

try:
    c = 2*M_g/Q_t0
    delta = (-Vao)**2 - 4*c
    x1 = np.round( (Vao-delta**0.5)/2 , decimals = 2)
    x2 = np.round( (Vao+delta**0.5)/2 , decimals = 2)
except:
    print("##   ATENÇÃO!   ##")
    raise Exception("ERRO: ESCALONAR MELHOR A PROTENSÃO!")
        
x1_anc = np.round( x1 - l_bpt , decimals = 2 )
x2_anc = np.round( x2 + l_bpt , decimals = 2 )

for i in range(len(x1_anc)):
    if x1_anc[i] < 0:
        x1_anc[i] = 0
    if x2_anc[i] > Vao:
        x2_anc[i] = Vao
        
tabela2 = pd.DataFrame([M_g, x1,x2, x1_anc,x2_anc])
tabela2 = tabela2.transpose()
tabela2.rename(columns={0:'Mg',1:'x1',2:'x2',3:'x1-lbpt',4:'x2+lbpt'},inplace = True)
display(tabela2)

,Mg,x1,x2,x1-lbpt,x2+lbpt
0,4094.5,71.93,1003.07,0.00,1075.00
1,10736.1,221.72,853.28,86.39,988.61


# Perdas Progressivas da Força de Protensão:

## Fluência do Concreto:

### Fluência Rápida Irreversível

In [19]:
%%render params

# VER ITEM 12.3.3 da NBR6118:2014
# Valores para adotar:
# s = 0.38 para concreto de cimento CP-III e CP-IV
# s = 0.25 para concreto de cimento CP-I e CP-II
# s = 0.20 para concreto de cimento CP-V
s = 0.20
t_0 = 10
t_F = 18250


beta_0 = exp(s*(1-(28/t_0)**0.5))
beta_F = exp(s*(1-(28/t_F)**0.5))

phi_a = 0.8*(1 - (beta_0/beta_F))

<IPython.core.display.Latex object>

In [20]:
if s == 0.38:
    print("Adotado s = 0.38 para concreto de cimento CP-III ou CP-IV")
elif s == 0.25:
    print("Adotado s = 0.25 para concreto de cimento CP-I ou CP-II")
else:
    print("Adotado s = 0.20 para concreto de cimento CP-V (ARI) ")

Adotado s = 0.20 para concreto de cimento CP-V (ARI) 


### Fluência Lenta Irreversível:

In [21]:
%%render params 2

U = 55 #umidade média do ar

phi_c1 = 4.45 - 0.035*U # para abatimentos entre 5cm e 9cm

gamma = 1 + exp(-7.8 + 0.1 * U)

u_ar = ( 2*h + 2*b )

h_fic = gamma*(2*A_c)/u_ar

phi_c2 = (42+h_fic)/(20+h_fic)

phi_fF = phi_c1*phi_c2

h_fm = h_fic/100 # espessura fictícia em metros

A_beta_f = 42*h_fm**3 - 350*h_fm**2 + 588*h_fm + 113
B_beta_f = 768*h_fm**3 - 3060*h_fm**2 + 3234*h_fm - 23
C_beta_f = -200*h_fm**3 + 13*h_fm**2 + 1090*h_fm + 183
D_beta_f = 7579*h_fm**3 - 31916*h_fm**2 + 35343*h_fm + 1931

beta_f_t0 = (t_0**2 + A_beta_f*t_0 + B_beta_f )/(t_0**2 + C_beta_f*t_0 + D_beta_f)
beta_f_tF = (t_F**2 + A_beta_f*t_F + B_beta_f )/(t_F**2 + C_beta_f*t_F + D_beta_f) 

<IPython.core.display.Latex object>

### Fluência Lenta Reversível:

In [22]:
%%render params

phi_dF = 0.4
beta_d = (t_F-t_0+20)/(t_F-t_0+70)

<IPython.core.display.Latex object>

### Fluência Total:

In [23]:
%%render 2 params
phi_t = phi_a + phi_fF*(beta_f_tF - beta_f_t0) + phi_dF * beta_d

<IPython.core.display.Latex object>

### Retração do Concreto:

In [24]:
%%render 2 params

epsilon_1s = (-8.09 + U/15 - U**2/2284 + U**3/133765 - U**4/7608150)*10**-4

epsilon_2s = (33+2*h_fic)/(20.8 + 3*h_fic)

A_beta_s = 40  
B_beta_s = 116*h_fm**3 - 282*h_fm**2 + 220*h_fm - 4.8  
C_beta_s = 2.5*h_fm**3 - 8.8*h_fm + 40.7  
D_beta_s = -75*h_fm**3 + 585*h_fm**2 + 496*h_fm - 6.8  
E_beta_s = -169*h_fm**4 + 88*h_fm**3 + 584*h_fm**2 - 39*h_fm + 0.8  

beta_s_t0 = ((t_0/100)**3 + A_beta_s*(t_0/100)**2 +  B_beta_s*(t_0/100))/((t_0/100)**3 + C_beta_s*(t_0/100)**2 +  D_beta_s*(t_0/100) + E_beta_s)

beta_s_tF = ((t_F/100)**3 + A_beta_s*(t_F/100)**2 +  B_beta_s*(t_F/100))/((t_F/100)**3 + C_beta_s*(t_F/100)**2 +  D_beta_s*(t_F/100) + E_beta_s)

epsilon_cs = epsilon_1s*epsilon_2s*(beta_s_tF - beta_s_t0)

<IPython.core.display.Latex object>

## Relaxação do Aço de Protensão:

In [25]:
a = np.zeros([3])
print(a)
print(a.size)

[0. 0. 0.]
3


In [26]:
relax = np.zeros([nS,7])

relax[:,1] = esforcos[:,0]
for i in range(nS):
    relax[i,0] = i+1
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[i,1] >= x1_anc[j] and relax[i,1] <= x2_anc[j]):
            relax[i,2] = round(sigma_p[j],4)
            break
            
relax[:,3] = np.round(relax[:,2]/f_ptk,decimals=4)
relax[:,4] = np.round(calcpsi(relax[:,3]),decimals=4)
relax[:,5] = np.round(relax[:,4]*2.5,decimals=4)
relax[:,6] = np.round(-log(1-relax[:,5]),decimals=4)
    
tabela3 = pd.DataFrame(relax)
tabela3.rename(columns={0:'Seção',1:'Pos',2:'Sigma,p',3:'Sigma,p/fptk',4:'\u03A81000',5:'\u03A8,inf',6:'X,inf'},inplace = True)
display(tabela3)

,Seção,Pos,"Sigma,p","Sigma,p/fptk",Ψ1000,"Ψ,inf","X,inf"
0,1.0,0.0,145.20,0.7642,0.0314,0.0785,0.0818
1,2.0,53.8,145.20,0.7642,0.0314,0.0785,0.0818
2,3.0,107.5,143.18,0.7536,0.0304,0.0760,0.0790
3,4.0,161.2,143.18,0.7536,0.0304,0.0760,0.0790
4,5.0,215.0,143.18,0.7536,0.0304,0.0760,0.0790
5,6.0,268.8,143.18,0.7536,0.0304,0.0760,0.0790
6,7.0,322.5,143.18,0.7536,0.0304,0.0760,0.0790
7,8.0,376.2,143.18,0.7536,0.0304,0.0760,0.0790
8,9.0,430.0,143.18,0.7536,0.0304,0.0760,0.0790
9,10.0,483.8,143.18,0.7536,0.0304,0.0760,0.0790


## Cálculo das Perdas Progressivas:

In [27]:
progressivas = np.zeros([nS,4])

alpha_p = E_p/E_cs

for i in range(nS):
    pos_i = relax[i,1]
    momento = 0.5*Q_t0*pos_i*(Vao-pos_i)
    for j in range(qntCordoalhas.size-1,-1,-1):
        if(relax[i,1] >= x1_anc[j] and relax[i,1] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            
            break
        else:
            sigma_pi = 0
            Npi = 0
            Mpi = 0
            Wcpi = 0
            e_pp = 0
            A_pp = 0
        
    if (Npi != 0):
        progressivas[i,0] = round(Npi/A_c + Mpi/Wcpi + momento/Wcpi,2)
        K_p = alpha_p*A_pp*(1/A_c + e_pp/Wcpi)
        progressivas[i,1] = round (((progressivas[i,0]*phi_t/E_cs + epsilon_cs - sigma_pi/E_p*relax[i,6])/(1+relax[i,6]+K_p*(1+0.5*phi_t)))*E_p,2)
        progressivas[i,2] = -round(progressivas[i,1]*A_pp,2)
        progressivas[i,3] = round(progressivas[i,2]*e_pp,2)
        
delta_Np_max = np.max(progressivas[i,2])
delta_Mp_max = np.max(progressivas[i,3])

tabela4 = pd.DataFrame(progressivas)
tabela4.rename(columns={0:'Sigma,c,t0',1:'\u0394Sigma,p',2:'\u0394Np',3:'\u0394Mp'},inplace = True)
display(tabela4)

,"Sigma,c,t0","ΔSigma,p",ΔNp,ΔMp
0,-0.88,-33.28,333.13,12492.38
1,-0.85,-32.74,327.73,12289.88
2,-1.08,-35.48,507.36,17757.60
3,-1.06,-35.13,502.36,17582.60
4,-1.04,-34.79,497.50,17412.50
5,-1.03,-34.61,494.92,17322.20
6,-1.01,-34.26,489.92,17147.20
7,-1.00,-34.09,487.49,17062.15
8,-0.99,-33.92,485.06,16977.10
9,-0.99,-33.92,485.06,16977.10


# Tensões Normais nas Bordas:

In [28]:
tensaobor = np.zeros((nS,5))

tensaobor[:,0] = relax[:,1]

for i in range(nS):
    for j in range(qntCordoalhas.size-1,-1,-1):
        if(relax[i,1] >= x1_anc[j] and relax [i,1] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[i,2]
    Mpx = Mpi + progressivas[i,3]
    tensaobor[i,1] = round( -abs(Npx/A_c) + abs(Mpx/W_s) - abs(esforcos[i,1]/W_s),3)
    
    if tensaobor[i,1] > -sigma_ccF:
        tensaobor[i,2] = 0
    else:
        tensaobor[i,2] = np.nan
    
    
    tensaobor[i,3] = round(-abs(Npx/A_c) -abs(Mpx/W_i) + abs(esforcos[i,1]/W_i) , 3)
    if tensaobor[i,3] < -sigma_ctF:
        tensaobor[i,4] = 0
    else:
        tensaobor[i,4] = np.nan
    

tabela5 = pd.DataFrame(tensaobor)
tabela5.rename(columns={0:'Pos (cm)',1:'Sigma,s',2:'OK',3:'Sigma,i',4:'OK'},inplace = True)
display(tabela5)

print("Limite de tensão na borda superior: {0:.2f} kN/cm²".format(-sigma_ccF))
print("Limite de tensão na borda inferior: {0:.2f} kN/cm²".format(-sigma_ctF))

,Pos (cm),"Sigma,s",OK,"Sigma,i",OK
0,0.0,0.298,0.0,-0.845,0.0
1,53.8,-0.133,0.0,-0.407,0.0
2,107.5,-0.460,0.0,-0.271,0.0
3,161.2,-0.800,0.0,0.074,0.0
4,215.0,-1.094,0.0,0.373,0.0
5,268.8,-1.344,0.0,0.627,NaN
6,322.5,-1.547,0.0,0.833,NaN
7,376.2,-1.706,0.0,0.994,NaN
8,430.0,-1.819,0.0,1.108,NaN
9,483.8,-1.887,0.0,1.178,NaN


Limite de tensão na borda superior: -2.00 kN/cm²
Limite de tensão na borda inferior: 0.42 kN/cm²


## Bordas Inferiores:

# Armaduras Passivas:

## Armadura da Cunha de Tração:

In [29]:
sigma_cs0 = -10000
pos = -1
for i in range (0,nS):
    if sigma_cs0 < tensaobor[i,1]:
        sigma_cs0 = tensaobor[i,1]
        pos = i

sigma_ci0 = tensaobor[pos,3]

print("Seção crítica de cálculo: seção",pos)

Seção crítica de cálculo: seção 0


In [30]:
%%render 2

x = sigma_cs0/(sigma_cs0 - sigma_ci0) * h

S_x = b*x**2/2

R_ct = sigma_cs0/x*S_x

A_st = R_ct/25.00

phi_st = 1.6

N_st = ceil(A_st/((phi_st/2)**2*pi))

<IPython.core.display.Latex object>

## Pré-Alongamento de Leito:

In [33]:
prealong = np.zeros([nS,5])

for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[i,1] >= x1_anc[j] and relax[i,1] <= x2_anc[j]):
                sigma_pw = sigma_p[j]
                Npw = N_p[j]
                Mpw = M_p[j]
                Wcpw = W_cp[j]
                break
    prealong[i,0] = Npw + progressivas[i,2]
    prealong[i,1] = Mpw + progressivas[i,3]
    prealong[i,2] = round(prealong[i,0]/A_c + prealong[i,1]/Wcpw ,2)
    prealong[i,3] = round( sigma_pw + progressivas[i,1] - alpha_p*prealong[i,2] ,2)
    prealong[i,4] = round(prealong[i,3]/E_p,5)



tabela7 = pd.DataFrame(prealong)
tabela7.rename(columns={0:'Np,F',1:'Mp,F',2:'Sigma,cpF', 3:'Sigma,PN',4:'e,pn'},inplace = True)
display(tabela7)

,"Np,F","Mp,F","Sigma,cpF","Sigma,PN","e,pn"
0,-1120.322,-42012.07,-0.68,116.19,0.00581
1,-1125.722,-42214.57,-0.68,116.73,0.00584
2,-1540.114,-53903.99,-0.86,113.10,0.00566
3,-1545.114,-54078.99,-0.86,113.45,0.00567
4,-1549.974,-54249.09,-0.86,113.79,0.00569
5,-1552.554,-54339.39,-0.86,113.97,0.00570
6,-1557.554,-54514.39,-0.87,114.38,0.00572
7,-1559.984,-54599.44,-0.87,114.55,0.00573
8,-1562.414,-54684.49,-0.87,114.72,0.00574
9,-1562.414,-54684.49,-0.87,114.72,0.00574


## Armadura Passiva de Flexão:

![formulario](img/formulario.png)

In [45]:
d_p = np.zeros_like(e_p_i) 
A_s = np.zeros([nS,8])

d_p = e_p_i + h/2  # braço de alavanca da armadura ativa
d_s = h-5          # braço de alavanca da armadura passiva

for i in range(nS):
    for j in range(e_p_i.size-1,-1,-1):
        if(relax[i,1] >= x1_anc[j] and relax[i,1] <= x2_anc[j]):
            sigma_pm = sigma_p[j]
            Npm = N_p[j]
            Mpm = M_p[j]
            Wcpm = W_cp[j]
            A_pm = A_p[j]
            d_sm = d_s
            d_pm = d_p[j]
            break
    Md = esforcos[i,1]*1.4
    retorno = linha_neutra_cp_ret(b,h,f_c,A_pm,prealong[i,4],d_pm,d_sm,Md)
    A_s[i,0] = int(Md)
    A_s[i,1] = retorno[0]
    A_s[i,2] = round(retorno[0]/d_sm,3)
    A_s[i,3] = retorno[2]
    A_s[i,5] = retorno[1]
    A_s[i,4] = retorno[4]
    A_s[i,6] = retorno[5]
    A_s[i,7] = round((0.8*b*f_c*retorno[0]-retorno[3])/retorno[4],2)

tabela8 = pd.DataFrame(A_s)
tabela8.rename(columns={0:'Md', 1:'x',2:'x/d',3:'eps,s',4:'sigma,s',5:'eps,p',6:'sigma,p',7:'As'},inplace = True)
display(tabela8)

,Md,x,x/d,"eps,s","sigma,s","eps,p","sigma,p",As
0,0.0,1.93,0.019,0.17782,43.48,0.16492,148.7,-30.78
1,44996.0,7.96,0.080,0.04048,43.48,0.04134,148.7,-20.01
2,85181.0,15.32,0.153,0.01935,43.48,0.02158,148.7,-21.52
3,120642.0,20.65,0.206,0.01345,43.48,0.01643,148.7,-12.00
4,151433.0,25.51,0.255,0.01022,43.48,0.01363,148.7,-3.31
5,177482.0,29.81,0.298,0.00824,43.48,0.01190,148.7,4.38
6,198755.0,33.48,0.335,0.00695,43.48,0.01080,148.7,10.94
7,215305.0,36.44,0.364,0.00611,43.48,0.01006,148.7,16.23
8,227149.0,38.61,0.386,0.00556,43.48,0.00960,148.7,20.10
9,234252.0,39.95,0.400,0.00526,43.48,0.00933,148.7,22.50


# Dimensionamento ao Esforço Cortante:

## Verificação da Diagonal Comprimida:

In [49]:
%%render

V_d = V_F * 1.4
tau_wd = V_d/((b-e_eps)*d_sm)
tau_wu = 0.27*(1-f_ck/25)*f_ck/1.4


<IPython.core.display.Latex object>

In [50]:
print("Tensão da biela: {0:.2f}%".format(tau_wd/tau_wu*100))

if(tau_wd/tau_wu>1.0):
    raise Exception("ERRO: ESMAGAMENTO DA BIELA COMPRIMIDA!")    

Tensão da biela: 36.17%


## Cálculo da Armadura de Corte:

### Armadura Mínima:

In [51]:
%%render short

rho_min = 0.2*f_ctm/f_yk
A_sws = rho_min*b

s = 10

A_sw_min = A_sws*s

nr = 2

phi_min = 2*(A_sw_min/(pi*nr))**0.5


<IPython.core.display.Latex object>

### Armadura Calculada:

In [53]:
i = int((nS+1)/2)
M_pf = prealong[i,1]
N_pf = prealong[i,0]

In [54]:
%%render 5

sigma_ciF = N_pf/A_c + M_pf/W_i
M_0 = -sigma_ciF*W_i
M_d_max = 1.4*M_F

V_c0 = 0.6*f_ctd*b*d_sm
V_c = (1+M_0/M_d_max)*V_c0

V_sw = 1.4*V_F - V_c

A_sws1 = V_sw/(0.9*d_sm*f_yd)

s = 10

A_sws = A_sws1*s

nr = 2
phi_calc = 2*(A_sws/(pi*nr))**0.5

<IPython.core.display.Latex object>

## Armadura de Introdução da Força de Protensão:

In [55]:
# VERIFICAR SE ESTA É MESMO A FORÇA DE PROTENSÃO INSERIDA NO APOIO

N_p1 = -N_p[0]

In [56]:
%%render

A_swi = (0.25*N_p1)/25.00

<IPython.core.display.Latex object>

Esta armadura deve ser distribuída num trecho equivalente à 70% do comprimento de regularização.

In [57]:
%%render

l_d = l_p*0.7

<IPython.core.display.Latex object>

# Cálculo da Flecha:

In [81]:
delta_p = np.zeros((e_p_i.size,3))

delta_pF = 0

delta_p[:,0] = A_p
delta_p[:,1] = x1_anc


for i in range(qntCordoalhas.size):
    if(i != 0):
        delta_p[i,0] = delta_p[i,0] - delta_p[i-1,0]
        Momen = M_p[i] - M_p[i-1]
    else:
        Momen = M_p[i]
        
    if(delta_p[i,1] == 0):
        delta_p[i,2] = round(Momen*Vao**2/(8*E_cs*I_c),2)
    else:
        delta_p[i,1] = round((x1_anc[i]+x1[i])/2,1)
        delta_p[i,2] = round( (Momen*(Vao**2-3*delta_p[i,1]**2-(Vao/2)**2)*(Vao/2))
                             /(6*E_cs*I_c*Vao) ,2)
    delta_pF += delta_p[i,2]



tabela9 = pd.DataFrame(delta_p)
tabela9.rename(columns={0:'Ap',1:'b',2:'Delta'},inplace = True)
display(tabela9)

N_pp = N_p[len(qntCordoalhas)-1]
M_pp = M_p[len(qntCordoalhas)-1]

if (delta_p[0][0]) < 0:
    raise Exception("ERRO")

,Ap,b,Delta
0,10.01,0.0,-0.64
1,4.29,154.1,-0.09


In [82]:
%%render
delta_pF

<IPython.core.display.Latex object>

## Momento de Fissuração:


In [83]:
%%render 2

# COEFICIENTE ALPHA T:
# alpha_t = 1.2 para seções T ou duplo T
# alpha_t = 1.3 para seções I ou T invertido
# alpha_t = 1.5 para seções retangulares

alpha_t = 1.2

M_r = (alpha_t*0.7*f_ctm - N_pp/A_c)* I_c / y_i - M_pp

<IPython.core.display.Latex object>

## Inércia no Estádio II:

In [84]:
## CALCULOS VALIDOS SOMENTE PARA SEÇÕES RETANGULARES:

alpha_s = E_s/E_cs
alpha_p = E_p/E_cs
As_II = A_s[int((nS-1)/2),7]
Ap_II = A_p[-1]
dp_II = d_p[-1]
ds_II = d_s

I_xc = lambda x: b*x**3/3
I_2 = lambda x: I_xc(x) + alpha_s*As_II*(ds_II-x)**2 + alpha_p*Ap_II*(dp_II-x)**2
S_x = lambda x: b*x**2/2
PN = prealong[int((nS-1)/2),3]*A_p[-1]

equacao = lambda x: I_2(x)/(S_x(x) - alpha_s*As_II*(ds_II-x) - alpha_p*Ap_II*(dp_II-x) ) - (M_ELS-PN*(dp_II-x))/PN
X_LN = fsolve(equacao,[h/2])[0]
I_II = I_2(X_LN)

if (I_II > I_c):
    I_II = I_c
    
print("Altura da linha neutra: {0:.2f} cm".format(X_LN))
print("Inércia da seção no estádio II: {0:.0f} cm⁴".format(I_II))

Altura da linha neutra: 67.48 cm
Inércia da seção no estádio II: 3858750 cm⁴


## Inércia Equivalente:

In [85]:
%%render

if (M_ELS > M_r):
    I_eq = (M_r/M_ELS)**3*I_c + (1-(M_r/M_ELS)**3)*I_II

if (M_ELS < M_r):
    I_eq = I_c

<IPython.core.display.Latex object>

## Flecha Diferida Final:

In [86]:
%%render 2

delta_i = (delta_pF + 5/384*(Q_ELS*Vao**4)/(E_cs*I_eq))*(1+phi_t)

<IPython.core.display.Latex object>